# Beispiel: Arbeiten mit Webseiten als Quellen

Viele Datenbank verfügen über eine Webseite, die eine Textsuche erlaubt. Das Result einer solchen Suche kann über das Python Paket beautifulsoup ausgelesen und in einen Dataframe geschrieben werden. 
Aufgrund dieser Daten kann dann weiter gearbeitet werden, sei es mit regulären Ausdrücken oder NLTK. 

In [32]:
import json
import re
import pandas as pd

import requests
from bs4 import BeautifulSoup

## Suchanfrage I: isisCB explore

Auf der Webseite für das isisCB explore Tool der "Isis Bibliography of the History of Science" werden Suchergebnisse über eine Link-Liste dargestellt. 

In der Liste werden einzelne Ergebnisse in paragraphen sortiert (mit dem tag `<p>`) . Wir generieren eine Unterliste aller paragraphen die das Word `Article` oder `Book` enthalten. 

In [33]:
url2 = 'https://data.isiscb.org/isis/?q=copernicus&models=isisdata.citation&sort_order_citation=publication_date_for_sort&sort_order_dir_citation=descend&sort_order_dir_authority=ascend&selected_facets=citation_subject_ids_exact:CBA000021037'

r2  = requests.get(url2)

data2 = r2.text

soup2 = BeautifulSoup(data2,'lxml')

In [34]:
baseURL = 'https://data.isiscb.org/'

Suche alle Paragraphen, die im Text das Wort Article oder Book enthalten.

In [35]:
plist = soup2.findAll('p')
queListe = [x for x in plist if "Article" in x.text or "Book" in x.text]

Für alle gefundenen Datensätze müssen wir nun einem weiteren Link folgen. Dieser wird über den tag `<a>` gesucht, mit der Option `href=True`.
Mit dem requests Paket wird eine Anfrage `GET` an die URL gesendet, die sich aus der Basis-URL und dem gefundenen Link zusammensetzt. 

Das Resultat wird in eine Soup verwandelt. In der neuen Webseite werden die Informationen zu einem Artikel oder Buch über das Element `div` mit der Klasse `class=col-sm 5` dargestellt. 
In einem solchen Element stehen die Informationen wieder in Paragraphen, die in eine Untermenge eingetragen werden.

In [36]:
resultList = []
for x in queListe:
    links = x.findAll('a',href=True)
    for link in links:
        scndLevel = baseURL + link['href']
        scndRes = requests.get(scndLevel)
        dataTemp = scndRes.text
        soupTemp = BeautifulSoup(dataTemp,'lxml')
        x = soupTemp.findAll('div',class_='col-sm-5')
        pListTemp = x[0].findAll('p')
        subList = []
        for parag in pListTemp:
            res = parag.text
            subList.append(res)
        resultList.append(subList)    

Mittels regex wird aus der Menge aller Resultate eine Menge von Dictionaries gebaut, vorraus wir einen sortierten Dataframe erhalten.

In [37]:
dictList = []
for i in range(len(resultList)):
    subDict = {}
    for k in range(len(resultList[i])):
        keys = re.findall('.+?(?=:)',resultList[i][k])
        value =re.findall('(?<=:).+',resultList[i][k]) 
        if keys:
            if value:
                subDict[keys[0]] = value[0]#resultList[i][k]
    dictList.append(subDict)

In [38]:
dfISISTemp = pd.DataFrame(dictList)
dfISISTemp.head(5)

,http,Adventures in the Bone Trade,Framing the Appearances in the Fifteenth Century,From Tūn to Toruń,In,Lee De Forest,"Review of ""Copernicus",Secrets of the Old One,The Status of Astronomy as a Science in Fifteenth-Century Cracow,Abstract,Authors & Contributors,Publication Date
0,//data.isiscb.org/isis/citation/CBB399918134/,NaN,"Alberti, Cusa, Regiomontanus, and Copernicus.",The Twists and Turns of the Ṭūsī-Couple.,Before Copernicus: The Cultures and Contexts ...,NaN,NaN,NaN,"Ibn al-Haytham, Peurbach, and Copernicus.","In 1984, Noel Swerdlow and Otto Neugebauer ar...",F. Jamil Ragep (Editor) ; Rivka Feldhay (Edit...,2017
1,//data.isiscb.org/isis/citation/CBB996745065/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This paper distinguishes four perspectives in...,Miguel de Asúa (Author) ;,2017
2,//data.isiscb.org/isis/citation/CBB778304283/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A page of notes in Copernicus’s hand shows th...,N.M. Swerdlow (Author) ;,2017
3,//data.isiscb.org/isis/citation/CBB593896461/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,It has long been recognized that Copernicus’ ...,F. Jamil Ragep (Author) ;,2016
4,//data.isiscb.org/isis/citation/CBB896915924/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The present paper follows the method introduc...,Gheorghe Stratan (Author) ;,2016


Um den Dataframe zu bereinigen, verwerfen wir alle Spalten in denen weniger als zwei Werte ungleich NaN sind. Zudem verwerfen wir alle Zeilen, die nur NaN Einträge enthalten.

In [39]:
dfISIS = dfISISTemp.dropna(axis=1,thresh=2)
dfISIS = dfISIS.dropna(axis=0,how='all')
dfISIS

,http,Abstract,Authors & Contributors,Publication Date
0,//data.isiscb.org/isis/citation/CBB399918134/,"In 1984, Noel Swerdlow and Otto Neugebauer ar...",F. Jamil Ragep (Editor) ; Rivka Feldhay (Edit...,2017
1,//data.isiscb.org/isis/citation/CBB996745065/,This paper distinguishes four perspectives in...,Miguel de Asúa (Author) ;,2017
2,//data.isiscb.org/isis/citation/CBB778304283/,A page of notes in Copernicus’s hand shows th...,N.M. Swerdlow (Author) ;,2017
3,//data.isiscb.org/isis/citation/CBB593896461/,It has long been recognized that Copernicus’ ...,F. Jamil Ragep (Author) ;,2016
4,//data.isiscb.org/isis/citation/CBB896915924/,The present paper follows the method introduc...,Gheorghe Stratan (Author) ;,2016
5,//data.isiscb.org/isis/citation/CBB917832903/,What moved Copernicus to switch from the time...,André Goddu (Author) ;,2016
6,//data.isiscb.org/isis/citation/CBB985320973/,Distortion of Scientific Terms by Supposed Mo...,Fritz Krafft (Author) ;,2016
7,//data.isiscb.org/isis/citation/CBB906472418/,Nicolaus Copernicus (1473-1543) is a pivotal ...,Owen Gingerich (Author) ;,2016
8,//data.isiscb.org/isis/citation/CBB847681863/,In 1454 Georg Peurbach taught astronomy at th...,Michela Malpangotto (Author) ;,2016
9,//data.isiscb.org/isis/citation/CBB706376729/,NaN,Robert S. Westman (Author) ;,2016


## Suchanfrage II: Newtons Gesammelte Werke 

Die Homepage des Newton Projects der University of Sussex, http://www.newtonproject.sussex.ac.uk/, bietet besser strukturierte Inhalte. 
Hier können die Informationen mit passenden Klassennamen sinnvoll ausgelesen werden. 

Die genauen Spezifikation sind als PDF verfügbar: http://www.newtonproject.sussex.ac.uk/resources/pdfs/techspec.pdf

In [40]:
url3 = 'http://www.newtonproject.sussex.ac.uk/prism.php?id=43'

r3  = requests.get(url3)

data3 = r3.text

soup3 = BeautifulSoup(data3,'lxml')

In [41]:
baseURLNewton = 'http://www.newtonproject.sussex.ac.uk'

Die Einträge werden in einer Table mit der Klasse `record` ausgegeben. 

In [42]:
allrecords = soup3.findAll('td',class_='record')

Der Title eines Eintrags:

In [43]:
allrecords[0].findAll('p',class_='title')[0].text

"Appointment of Edward Harley as keeper of the officers' diet [ie. caterer], succeeding Richard Millward."

den Author erhalten wir über 

In [44]:
allrecords[0].findAll('p',class_='author')[0].text

'Author: \n                            \n                                Isaac Newton\n                            \n                        '

ebenso verfahren wir mit metadata, source, links und primary_key. Wieder wird eine Liste mit Sub-Dictionaries erzeugt und daraus ein Dataframe gebaut. 

Um den normalisierten Volltext zu erhalten, folgen wir wieder einem Link auf die Unterseite und extrahieren dort den Text. Dieser ist eindeutig markiert mit der id `tei`
Über den Befehl `.text` erhalten wir den Text ohne die HTML-Struktur.

**Achtung:**

Das Abfragen von allen Einträgen dauert etwas länger. Daher werden unten nur die ersten 10 Einträge abgefragt (`[:10]`) um den Server der Uni Sussex nicht zu überlasten. Wenn die Texte mit NLTK weiter bearbeitet werden sollen, bietet es sich daher an, den resultierenden Dataframe als Pickle abzuspeichern. 

In [45]:
dictListNewton = []
for record in soup3.findAll('td',class_='record')[:10]:
    subDict = {}
    for keyClass in ['title', 'author','metadata','source','primary_key']:
        try:
            value = record.findAll('p',class_=keyClass)[0].text
            subDict[keyClass] = value
        except:
            subDict[keyClass] = None
    links = record.findAll('a',href=True,text='Normalized\xa0Text')
    if links:
        scndLevel = baseURLNewton + links[0]['href']
        scndRes = requests.get(scndLevel)
        dataTemp = scndRes.text
        soupTemp = BeautifulSoup(dataTemp,'lxml')
        text = soupTemp.findAll('div',id='tei')
        subDict['norm_text'] = text[0]
    dictListNewton.append(subDict)    

In [46]:
dfNewton = pd.DataFrame(dictListNewton)
dfNewton.head(3)

,author,metadata,primary_key,source,title
0,Author: \n \n ...,"Metadata: 8 June 1622., \n ...",None,None,Appointment of Edward Harley as keeper of the ...
1,Author: \n \n ...,"Metadata: 1659-early 1660s, in English, \n ...",None,None,Pierpont Morgan Notebook
2,Author: \n \n ...,"Metadata: early-mid 1660s, in Greek, Latin an...",None,None,'Quæstiones quædam Philosophiæ' ('Certain Phil...


In [47]:
dfNewton.shape

(10, 5)

### Speichern und Laden von Dataframes

Die Struktur von Dataframes bietet es an, diese als JSON zu speichern. Das erfolgt einfach mit df.to_json('Dateiname'). Über die normale Einleseroutine von JSONs kann dann der Dataframe aus der Datei wieder hergestellt werden. Die bereits vorhandene JSON Datei enthält alle 250 Einträge zu Newton.

In [48]:
# Save dataframe to json file on disk
#dfNewton.to_json('./newton_metadata_frame.json')

In [49]:
# Load by using
# with open('./newton_metadata_frame.json') as NewtonData:
#     newtonJson = json.load(NewtonData)
#     dfTest = pd.DataFrame(newtonJson)

# Geographische Daten

Antike Orte in dem Projekt https://pleiades.stoa.org/ können über eine API abgefragt werden: http://api.pleiades.stoa.org/

Zeige den aktuellen Zustand des Projekts

In [50]:
res = requests.get('http://api.pleiades.stoa.org/status')
res.json()

{'num_locations': 39462, 'num_names': 32064, 'num_places': 35761}

Erhalte Informationen zu einer bestimmten Stadt, kodiert durch ID.

In [51]:
res2 = requests.get('http://pleiades.stoa.org/places/423025/json')
jsonRes = res2.json()

In [52]:
jsonRes.keys()

dict_keys(['placeTypes', '@type', 'connectsWith', 'references', 'description', 'title', 'rights', 'created', 'history', 'review_state', '@context', 'uri', 'subject', 'contributors', 'names', 'id', 'provenance', 'bbox', 'features', 'details', 'locations', 'creators', 'type', 'reprPoint'])

In [53]:
dfRoma = pd.DataFrame([jsonRes])
dfRoma

,@context,@type,bbox,connectsWith,contributors,created,creators,description,details,features,...,placeTypes,provenance,references,reprPoint,review_state,rights,subject,title,type,uri
0,"{'title': 'dcterms:title', 'rights': 'dcterms:...",Place,"[12.486137, 41.891775, 12.486137, 41.891775]","[https://pleiades.stoa.org/places/303999556, h...","[{'name': 'DARMC', 'username': None}, {'name':...",2010-11-10T22:33:36Z,"[{'name': 'L. Quilici', 'username': None}, {'n...",The capital of the Roman Republic and Empire.,<p>The Barrington Atlas Directory notes: Roma/...,"[{'id': 'darmc-location-30635', 'geometry': {'...",...,"[urban, settlement]",Barrington Atlas: BAtlas 43 B2 Roma,"[{'otherIdentifier': ' ', 'accessURI': '', 'sh...","[12.486137, 41.891775]",published,Copyright © The Contributors. Sharing and remi...,"[dare:major=1, dare:feature=major settlement, ...",Roma,FeatureCollection,https://pleiades.stoa.org/places/423025


Zeige verknüpfte Straßen zu dem Hauptort. Hierzu wird für jede Adresse in der Liste `dfRoma[connectsWith]` eine Anfrage gestellt und aus dem resultierenden JSON der Titel ausgelesen. 

In diesem Beispiel werden nur die ersten 20 Links abgefragt. 

In [54]:
connectedStreets = []
for i in (dfRoma['connectsWith'].iloc[0])[:20]:
    res = requests.get(i + '/json')
    try:
        jsonRes = res.json()
        title = jsonRes['title']
        connectedStreets.append(title)
    except:
        pass

In [55]:
connectedStreets

['Via Portuensis?',
 'Via Aemilia Scauri',
 'Via Latina',
 'Via Cornelia',
 'Via Nomentana',
 'Via Ostiensis',
 'Via Ardeatina',
 'Via Tiburtina',
 'Via Collatina',
 'Via Appia',
 'Via Clodia',
 'Via Aurelia',
 'Via Cassia',
 'Via Salaria',
 'Via Flaminia',
 'Via Praenestina',
 'Tiberis (river)',
 'Via Triumphalis',
 'Via Sublacensis']